In [10]:
__version__ = '0.3'
__author__  = "Robert Matern (r.matern@stud.uni-hannover.de)"
__date__    = ''
__url__     = ''
__copyright__ = "(C) 2015 Robert Matern"

In [11]:
import numpy as np
import pandas as pd
import matplotlib as plt

#Dynamisches Programm

Die normale Modellformulierung des Auftragsannahmeproblems im Revenue Management von Instandhaltungsprozessen:

$$V(\textbf{c}, t) = \sum_{j \in \mathcal{J}}p_{j}(t)\max[V(\textbf{c}, t-1), r_{j} + V(\textbf{c}-\textbf{a}_j, t-1)] + p_{0}(t)V(\textbf{c}, t-1) $$

$$= V(\textbf{c}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t) \max[r_j - V(\textbf{c}, t-1) + V(\textbf{c}-\textbf{a}_j, t-1), 0]$$

Erweiterung um einen Lagerbestand an Fertigerzeugnissen in Form von Produkten $j \in \mathcal{J}$:


$$V(\textbf{c}, \textbf{y}, t) = \sum_{j \in \mathcal{J}}p_{j}(t)\max[V(\textbf{c}, \textbf{y}, t-1), r_{j} + V(\textbf{c}-\textbf{a}_j, \textbf{y}, t-1), r_{j} + V(\textbf{c}, \textbf{y}-\textbf{s}_j, t-1)] + p_{0}(t)V(\textbf{c}, \textbf{y}, t-1) $$

$$= \sum_{j \in \mathcal{J}}p_{j}(t)V(\textbf{c}, \textbf{y}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t)\max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}-\textbf{a}_j, \textbf{y}, t-1),0] +  \sum_{j \in \mathcal{J}}p_{j}(t)\max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}, \textbf{y}-\textbf{s}_j, t-1),0] + p_{0}(t)V(\textbf{c}, \textbf{y}, t-1) $$

$$= \sum_{j \in \mathcal{J}}p_{j}(t)V(\textbf{c}, \textbf{y}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t)[\max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}-\textbf{a}_j, \textbf{y}, t-1),0] + \max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}, \textbf{y}-\textbf{s}_j, t-1),0]] + p_{0}(t)V(\textbf{c}, \textbf{y}, t-1) $$

$$= V(\textbf{c}, \textbf{y}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t)[\max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}-\textbf{a}_j, \textbf{y}, t-1),0] + \max[r_{j} - V(\textbf{c}, \textbf{y}, t-1) + V(\textbf{c}, \textbf{y}-\textbf{s}_j, t-1),0]]$$

## Algorithmus

In [12]:
solutions = {}

def DP_Storage(solutions, conditions, products, resources, capacities, consumtions, times, probs, stocks, shifts):
    '''Berechnung der Erwartungswerte des Auftragsannahmeproblems inkl. Lagerhaltung.'''
    
    # Leere Wert für den Erwartungswert des aktuellen Systemzustands.
    value = 0
    
    #Aktueller Systemzustand wird generiert.
    condition = np.append(capacities[1:], stocks[1:])
    condition = np.append(condition, times[0])
    
    #Aktueller Systemzustand wird in dem 'np.array' aller möglichen Systemzustände gesucht.
    state = np.where((conditions == condition).all(axis=1))[0][0]
    
    # Memofunktion: Die Rechnung wird nur fortgeführt, sofern es nicht schon berechnet wurde.
    if state not in solutions:
                
        # Sofern es sich nicht um einen Endknoten des Entscheidungsbaums handelt,
        # werden folgende Schritte eingeleitet:
        if times[0]!=0:
            # Der Erwartungswert t-1 ohne Akeptanz wird ermittelt und im Wert 'reject' gespeichert.
            reject = DP_Storage(solutions, conditions, products, resources, capacities, consumtions, times[1:], probs, stocks, shifts)
            # Für den Erwartungswert t-1 mit Akzeptanz werden Numpy-Arrays in der Länge der Anzahl an Produkten erstellt.
            accept = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            max_order = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            # Da hier auch ein Lagerhaltungbestand berücksichtigt ist, wird auch hierführ Numpy-Arrays erstellt. 
            accept_storage = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            max_order_storage = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            
            # For-Schleife über alle Produkte, sofern die Kapazitäten keinen negativen Werte annehmen.
            for j in products[1:]:
                change = capacities-consumtions[j]
                change_stock = stocks-shifts[j]
                if np.all((change) >= np.float32(0)):
                    # Initialisierung der Rechnung für t-1 mit Akeptanz jeweils für ein Produkt j.
                    condition_accept = np.append(capacities[1:], stocks[1:])
                    condition_accept = np.append(condition_accept, times[0])
                    state_accept = np.where((conditions == condition_accept).all(axis=1))[0][0]
                    if state_accept not in solutions:
                        accept_j = DP_Storage(solutions, conditions, products, resources, change, consumtions, times[1:], probs, stocks, shifts)
                        max_order[j-1] = max(revenues[j]-reject+accept_j, 0)
                        accept[j-1] = probs[j][times[0]]*max_order[j-1]
                    else:
                        accept[j-1] = solutions[state_accept][1]                    
                
                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,st)=-∞, falls n[j] < 0.
                    accept[j-1] = np.float32(0)
                    
                if np.all((change_stock) >= np.float32(0)):
                    # Initialisierung der Rechnung für t-1 mit Akeptanz jeweils für ein Produkt j.
                    condition_accept_storage = np.append(capacities[1:], change_stock[1:])
                    condition_accept_storage = np.append(condition_accept_storage, times[0])
                    state_accept_storage = np.where((conditions == condition_accept_storage).all(axis=1))[0][0]
                    if state_accept_storage not in solutions:
                        accept_storage_j = DP_Storage(solutions, conditions, products, resources, capacities, consumtions, times[1:], probs, change_stock, shifts)
                        max_order_storage[j-1] = max(revenues[j]-reject+accept_storage_j, 0)
                        accept_storage[j-1] = probs[j][times[0]]*max_order_storage[j-1]
                    else:
                        accept[j-1] = np.float32(0)
                        accept_storage[j-1] = solutions[state_accept_storage][1]
                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,st)=-∞, falls n[j] < 0.
                    accept_storage[j-1] = np.float32(0)
                            
            # Summierung der Erwartungswerte ohne und mit Akzeptanz.
            value = np.float32(reject + np.sum(accept) + np.sum(accept_storage))
            # Für den aktuellen Systemzustand wird der Ertragswert und die opt. Politik in das Dict "solutions" gespeichert.
            opt_order = np.where((max_order == max(max_order)))[0][0]+1
            opt_order_storage = np.where((max_order_storage == max(max_order_storage)))[0][0]+1
            if opt_order >= opt_order_storage:
                solutions[state] = [conditions[state], value, opt_order, 'AA']
                print 'Systemzustand:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2], solutions[state][3]
            else:
                solutions[state] = [conditions[state], value, opt_order_storage, 'LE']
                print 'Systemzustand:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2],  solutions[state][3]
                
        # Sofern es sich um einen Endknoten des Entscheidungsbaums handelt, werden folgende Schritte eingeleitet:
        else:
            # Erwartungswert enspricht der Grenzbedingung V(c,0)=0, für n >= 0.
            value = np.float32(0)
            # Ein Endzustand wird mit einem Erwartungswert 0 in das Dict "solutions" gespeichert.
            solutions[state] = [conditions[state], value, 0, 'KA']
            print 'Systemzustand:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik:', solutions[state][2], solutions[state][3]
            #print 'Periode:', times[0], 'Res.Kapa:', capacities[1:], '#Lösungen:', len(solutions), 'Lösung:', value         
            
    # Memofunktion: Sofern das Ergebnis breits berechnet wurde, wird der Wert aus dem Dict "solutions" verwendet.
    else:
        value = solutions[state][1]
               
    return value

## Erstellung der Struktur als NetworkX-Graph

In [ ]:
import networkx as nx

def Structure_Storage(solutions, conditions, products, consumtions, revenues, stocks, shifts):
    '''Generierung der Stuktur der Problemstellung.'''
    
    # MultiDiGraph erstellen.
    graph=nx.MultiDiGraph()
    
    # Knoten für alle Lösungen (solutions) erstellen.
    for key in solutions.iterkeys():
        graph.add_node(key, label=solutions[key][0], value=solutions[key][1],
                   capacity=solutions[key][0][:-1], time=solutions[key][0][-1])
    
    # Endknoten zur Nutzung der NetworkX-Algorithmen anlegen.
    graph.add_node("end", label='end', value=0,
                   capacity=0, time=0)
    
    # Kanten erstellen.
    # Schleife über alle Lösungen.
    for i in solutions.iterkeys():
        # Handelt es sich um einen Zeitpunkt 0, dann wird der Systemzustand mit dem 'end'-Knoten verbunden.
        if solutions[i][0][-1] == 0:
            graph.add_edge(i, "end", key=0, weight=0, revenue=0, time=0)
        # Sonst führe die Schleife aus.
        else:
            # Aufgrund der differenzierten Auftragsannahme erfolgt eine Schleife über alle Produkte.
            for j in products:
                # Kapazitätsveränderung aufgrund der Anfragen nach Produkt 'j' wird erfasst.
                change = solutions[i][0][:-(len(stocks))]-consumtions[j][1:]
                # Keine negativen Kapazitätsbestände möglich.
                if np.all((change) >= np.float32(0)): 
                    # Zielzustand wird ermittelt. 
                    reduction = np.append(change, solutions[i][0][(len(consumtions)-1):-1])
                    reduction = np.append(reduction, solutions[i][0][-1]-1)
                    state = np.where((conditions == reduction).all(axis=1))[0][0]
                    # Handelt es sich um eine Ablehnung, dann wird folgende Kante erstellt.
                    if  j == 0:
                        graph.add_edge(i, state, key=0, weight=solutions[state][2], weight_goal=solutions[state][1], revenue=revenues[j], goal=solutions[state][0], time=solutions[i][0][-1])
                    # Handelt es sich um eine Annahme, dann wird nachfolgende Kante erstellt.
                    if  j != 0:
                        # Zielzustand zur Ermittlung der Opportunitätskosten wird ermittelt.
                        no_reduction = np.append(solutions[i][0][:-1], solutions[i][0][-1]-1)
                        state2 = np.where((conditions == no_reduction).all(axis=1))[0][0]
                        # Abfrage, ob der Revenue höher ist als die Opportunitätskosten.
                        if revenues[j] >= solutions[state2][1]-solutions[state][1]:
                            graph.add_edge(i, state, key=j, weight=revenues[j]-solutions[state2][1]+solutions[state][1], weight_goal=solutions[state][1], revenue=revenues[j], goal=solutions[state][0], time=solutions[i][0][-1])
                        else:
                            graph.add_edge(i, state, key=j, weight=0, weight_goal=solutions[state][1], revenue=revenues[j], goal=solutions[state][0], time=solutions[i][0][-1])
                
                change_storage = solutions[i][0][(len(products)-1):-1]-shifts[j][1:]
                if np.all((change_storage) >= np.float32(0)): 
                    # Zielzustand wird ermittelt. 
                    reduction_storage = np.append(solutions[i][0][:-len(stocks)], change_storage)
                    reduction_storage = np.append(reduction_storage, solutions[i][0][-1]-1)
                    state_storage = np.where((conditions == reduction_storage).all(axis=1))[0][0]
                    # Handelt es sich um eine Ablehnung, dann wird folgende Kante erstellt.
                    if  j == 0:
                        graph.add_edge(i, state_storage, key=0, weight=solutions[state_storage][2], weight_goal=solutions[state_storage][1], revenue=revenues[j], goal=solutions[state_storage][0], time=solutions[i][0][-1])
                    # Handelt es sich um eine Annahme, dann wird nachfolgende Kante erstellt.
                    if  j != 0:
                        # Zielzustand zur Ermittlung der Opportunitätskosten wird ermittelt.
                        no_reduction_storage = np.append(solutions[i][0][:-1], solutions[i][0][-1]-1)
                        state2_storage = np.where((conditions == no_reduction_storage).all(axis=1))[0][0]
                        # Abfrage, ob der Revenue höher ist als die Opportunitätskosten.
                        if revenues[j] >= solutions[state2_storage][1]-solutions[state_storage][1]:
                            graph.add_edge(i, state_storage, key=j, weight=revenues[j]-solutions[state2_storage][1]+solutions[state_storage][1], weight_goal=solutions[state_storage][1], revenue=revenues[j], goal=solutions[state_storage][0], time=solutions[i][0][-1])
                        else:
                            graph.add_edge(i, state_storage, key=j, weight=0, weight_goal=solutions[state_storage][1], revenue=revenues[j], goal=solutions[state_storage][0], time=solutions[i][0][-1])
    return graph

## Ermittlung der optimalen Politik

In [ ]:
def Best_Politic(graph, times, resources, start=None):
    '''Ermittlung der optimalen Politik ohne Berücksichtigung der Nachfrage.'''
    
    print 'Optimalen Politik zum Zeitpunkt "t" und unter Beachtung der Restkapazitäten "c[h]":', '\n'
        
    # List mit topologischer Sortierung des Graphen wird erstellt.
    list = nx.topological_sort(graph)
    
    # Leere Listen für die 'Pandas'-Tabellenspalten erstellen
    db_sol = []
    db_time = []
    db_res = []
    db_suc = []
    db_bp = []
    db_oc = []
    db_sucp = []
     
    # Schleife über alle Knoten der topologisch sortierten Liste.
    for node in list:
        # Listen für 'Pandas'-Tabellenspalten erstellen.
        db_sol.append(graph.node[node]['value'])
        db_time.append(graph.node[node]['time'])
        db_suc.append(graph.successors(node))
        # Ist die Schleife zum Endknoten der Liste gekommen, dann wird sie unterbrochen.
        if node == 'end':
            db_bp.append(0)
            db_oc.append(0)
            db_sucp.append(0)
            break
        # Ist die Schleife zu einem Endzeitpunkt gekommen, dann wird sie übersprungen.
        if graph.node[node]['time'] == 0:
            db_bp.append(0)
            db_oc.append(0)
            db_sucp.append(0)
            continue
        # Sonst werden die möglichen Auftragsannahmen des Knotens in eine Liste geschrieben.
        else:
            # Möglichen Auftragsannahmen des Knotens werden erstellt.
            array = []
            for suc in graph.successors(node):
                for order in graph.edge[node][suc].iterkeys():
                    array.append((node,suc,order,graph.edge[node][suc][order]['weight']))

                    # Boolescher Wert 'not_best_politic' zur Ermittlung des Pfads wird angelegt.
                    graph.edge[node][suc][order]['not_best_politic']=True
            
            # Sofern keine Auftragsannahme möglich ist, wird kein Wert in den Listen gespeichert.
            if not array:
                db_bp.append(np.nan)
                db_oc.append(np.nan)
                db_sucp.append(np.nan)
            # Sofern Auftragsannahmen möglich sind, wird die Option mit der besten Politik
            # (max. Revenue abzgl. der Opportunitätskosten) in die Listen gespeichert.
            else:
                maxweight = max(array, key=lambda x: x[3])
                db_bp.append(maxweight[2])
                db_oc.append(maxweight[3])
                db_sucp.append(maxweight[1])
                
                # Boolescher Wert 'not_best_politic' für den Pfad wird bestimmt.
                graph.edge[maxweight[0]][maxweight[1]][maxweight[2]]['not_best_politic']=False
    
    # Spalten für die Ressourcenkapazitäten erstellen.
    for res in resources[1:]:
        db_res.append('Res_Cap'+str(res))
    db_cap = []
    for c in range(len(db_res)):
        cap = []
        for n in list:
            if n == 'end':
                cap.append(0)
            else:
                cap.append(graph.node[n]['capacity'][c])
        db_cap.append(cap)

    # Tabelle erstellen                
    db = pd.DataFrame(index=list)
    db['Value'] = db_sol
    db['Successor'] = db_suc
    for i, res in enumerate(db_res):
        db[res] = db_cap[i]
    db['Time'] = db_time
    db['Best Order'] = db_bp
    db['Best Successor'] = db_sucp
    db['r[j]-OC[j,c,t]'] = db_oc
    
    
    # Tabelle ausgeben.
    db = db[:-1]
    db.to_csv('Table_Graph.csv')
    print db, '\n'
    
    # Optimaler Pfad wird mittels Dijkstra-Algorithmus geschrieben.
    path = nx.dijkstra_path(graph, source=list[0], target=list[-1], weight='not_best_politic')[:-1]
    
    # Tabelle erstellen.
    rev = [0] * len(times)
    dec = [0] * len(times)
    fr = [0] * len(times)
    to = [0] * len(times)
    for i, node in enumerate(path[:-1]):
        for order in graph.edge[node][path[i+1]].iterkeys():
            if graph.edge[node][path[i+1]][order]['not_best_politic'] == False:
                rev[graph.node[node]['time']] = graph.edge[node][path[i+1]][order]['revenue']
                dec[graph.node[node]['time']] = order
                fr[graph.node[node]['time']] = path[i]
                to[graph.node[node]['time']] = path[i+1]

    df = pd.DataFrame(dec[::-1], index=times, columns = ['Opt. Politic'])
    #df.index.name = 'time'
    df['from'] = fr[::-1]
    df['to'] = to[::-1]
    df['Revenue'] = rev[::-1]
    df['Cum_Rev'] = df.Revenue.cumsum()
    df.to_csv('Table_Politic.csv')
    print df, '\n'
        
    return db, df

def Best_Politic_Demand(graph, times, resources, demands, start=None):
    '''Ermittlung der optimalen Politik mit Berücksichtigung der Nachfrage.'''
    
    print 'Optimalen Politik zum Zeitpunkt "t" und unter Beachtung der Restkapazitäten "c[h]" sowie der tatsächlich eintreffenden Nachfrage "d[j,t]":', '\n'
    
    # List mit topologischer Sortierung des Graphen wird erstellt.
    list = nx.topological_sort(graph)
    
    # Leere Listen für die 'Pandas'-Tabellenspalten erstellen
    db_sol = []
    db_time = []
    db_res = []
    db_suc = []
    db_bp = []
    db_oc = []
    db_sucp = []
     
    # Schleife über alle Knoten der topologisch sortierten Liste.
    for node in list:
        # Listen für 'Pandas'-Tabellenspalten erstellen.
        db_sol.append(graph.node[node]['value'])
        db_time.append(graph.node[node]['time'])
        db_suc.append(graph.successors(node))
        # Ist die Schleife zum Endknoten der Liste gekommen, dann wird sie unterbrochen.
        if node == 'end':
            db_bp.append(0)
            db_oc.append(0)
            db_sucp.append(0)
            break
        # Ist die Schleife zu einem Endzeitpunkt gekommen, dann wird sie übersprungen.
        if graph.node[node]['time'] == 0:
            db_bp.append(0)
            db_oc.append(0)
            db_sucp.append(0)
            continue
        # Sonst werden die möglichen Auftragsannahmen des Knotens in eine Liste geschrieben.
        else:
            # Möglichen Auftragsannahmen des Knotens werden erstellt.
            array = []
            for suc in graph.successors(node):
                for order in graph.edge[node][suc].iterkeys():
                    # Sofern keine Anfrage eintrifft, wird die mögliche Auftragsannahme nicht aufgenommen.
                    if demands[order][graph.edge[node][suc][order]['time']] == True:
                        array.append((node,suc,order,graph.edge[node][suc][order]['weight']))

                    # Boolescher Wert 'not_best_politic' zur Ermittlung des Pfads wird angelegt.
                    graph.edge[node][suc][order]['not_best_politic']=True
            
            # Sofern keine Auftragsannahme möglich ist, wird kein Wert in den Listen gespeichert.
            if not array:
                db_bp.append(np.nan)
                db_oc.append(np.nan)
                db_sucp.append(np.nan)
            # Sofern Auftragsannahmen möglich sind, wird die Option mit der besten Politik
            # (max. Revenue abzgl. der Opportunitätskosten) in die Listen gespeichert.
            else:
                maxweight = max(array, key=lambda x: x[3])
                db_bp.append(maxweight[2])
                db_oc.append(maxweight[3])
                db_sucp.append(maxweight[1])
                
                # Boolescher Wert 'not_best_politic' für den Pfad wird bestimmt.
                graph.edge[maxweight[0]][maxweight[1]][maxweight[2]]['not_best_politic']=False
    
    # Spalten für die Ressourcenkapazitäten erstellen.
    for res in resources[1:]:
        db_res.append('Res_Cap'+str(res))
    db_cap = []
    for c in range(len(db_res)):
        cap = []
        for n in list:
            if n == 'end':
                cap.append(0)
            else:
                cap.append(graph.node[n]['capacity'][c])
        db_cap.append(cap)

    # Tabelle erstellen                
    db = pd.DataFrame(index=list)
    db['Value'] = db_sol
    db['Successor'] = db_suc
    for i, res in enumerate(db_res):
        db[res] = db_cap[i]
    db['Time'] = db_time
    db['Best Order'] = db_bp
    db['Best Successor'] = db_sucp
    db['r[j]-OC[j,c,t]'] = db_oc
    
    
    # Tabelle ausgeben.
    db_demand = db[:-1]
    db_demand.to_csv('Table_Graph_Demand.csv')
    print db_demand, '\n'
    
    # Optimaler Pfad wird mittels Dijkstra-Algorithmus geschrieben.
    path = nx.dijkstra_path(graph, source=list[0], target=list[-1], weight='not_best_politic')[:-1]
    
    # Tabelle erstellen.
    rev = [0] * len(times)
    dec = [0] * len(times)
    fr = [0] * len(times)
    to = [0] * len(times)
    for i, node in enumerate(path[:-1]):
        for order in graph.edge[node][path[i+1]].iterkeys():
            if graph.edge[node][path[i+1]][order]['not_best_politic'] == False:
                rev[graph.node[node]['time']] = graph.edge[node][path[i+1]][order]['revenue']
                dec[graph.node[node]['time']] = order
                fr[graph.node[node]['time']] = path[i]
                to[graph.node[node]['time']] = path[i+1]

    df = pd.DataFrame(dec[::-1], index=times, columns = ['Opt. Politic'])
    #df.index.name = 'time'
    df['from'] = fr[::-1]
    df['to'] = to[::-1]
    df['Revenue'] = rev[::-1]
    df['Cum_Rev_Demand'] = df.Revenue.cumsum()
    df.to_csv('Table_Politic_Demand.csv')
    df_demand = df
    print df, '\n'

    return db_demand, df_demand

def Best_Path(graph, times, start=None):
    '''Ermittlung der optimalen Politik.'''
    list = nx.topological_sort(graph)
    if start == None:
        pol = nx.dijkstra_path(graph, list[0], "end", weight='weight_goal')
    else:
        pol = nx.dijkstra_path(graph, start, "end", weight='weight_goal')
    
    print "Kürzester Pfad:", pol[:-1], '\n'
    
    #### Funktioniert nicht, da er bei einem MultiDiGraph den Revenue überschreibt.
    #rev = [0] * len(times)
    #for i, node in enumerate(pol[:-1]):
    #    for order in graph.edge[node][pol[i+1]].iterkeys():
    #        rev[graph.node[node]['time']] = graph.edge[node][pol[i+1]][order]['revenue']
    #print 'Revenue je Buchungsperiode:', rev[::-1]
    #print 'Gesamt-Revenue:', sum(rev), '\n'
    
    return

In [ ]:
def Drawing(graph, db, df, db_demand, df_demand, size_x=5, size_y=5):
    '''Grafische Darstellung des Entscheidungsbaums.'''
    
    
    fig, ax = plt.subplots()
    plt.title("Verlauf des Revenues")
    plt.xticks(times)
    plt.yticks()
    
    #MaxValue = df['Cum_Rev'].max()
    #Text = 'Gesamtrevenue: ' + str(MaxValue)
    #MaxValue2 = df_demand['Cum_Rev'].max()
    #Text2 = 'Gesamtrevenue (Nachfrage): ' + str(MaxValue2)
    
    df['Cum_Rev'].plot(ax=ax, legend=True)
    df_demand['Cum_Rev_Demand'].plot(ax=ax, legend=True)
    
    #df['Cum_Rev'].plot(legend=True, sharex=True, sharey=True)
    #df_demand['Cum_Rev_Demand'].plot(legend=True, sharex=True, sharey=True)
    #plt.annotate(Text, xy=(1, MaxValue), xytext=(8, 0), 
    #             xycoords=('axes fraction', 'data'), textcoords='offset points')
    #plt.annotate(Text, xy=(1, MaxValue2), xytext=(8, 0), 
    #             xycoords=('axes fraction', 'data'), textcoords='offset points')
    plt.savefig('Revenue_Plot.png')
    
    
    
    size = len(nx.nodes(graph))
    if not size >= 100:
        draw = graph.copy()
        draw.remove_node('end')
        # Quelle: http://stackoverflow.com/a/11484144/4913569
        # ----
        # write dot file to use with graphviz
        # run "dot -Tpng test.dot >test.png"
        nx.write_dot(draw,'test.dot')

        # same layout using matplotlib with no labels
        pos=nx.graphviz_layout(draw,prog='dot')
        plt.figure(figsize=(size_x,size_y))
        plt.title("Entscheidungsbaum")
        nx.draw(draw,pos,with_labels=True, node_size=500)
        plt.savefig('Graph.png')
    else:
        raise RuntimeError('Plott der Grafik nicht möglich (Knotenanzahl: %i >= 100).' % size)
    return